In [1]:
import cv2
import os
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
import openslide
import libtiff
libtiff.libtiff_ctypes.suppress_warnings()

from mask_to_xml import *

In [2]:
tissue_kernel_1 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
tissue_kernel_2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

def get_otsu_mask(_img):
    _, _, v = cv2.split(cv2.cvtColor(_img, cv2.COLOR_RGB2YUV))
    v = cv2.medianBlur(v, 5)
    _, _mask = cv2.threshold(v, -1, 255, cv2.THRESH_OTSU)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=5)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=5)
    return _mask

def get_gray_mask(_img):
    v = cv2.cvtColor(_img, cv2.COLOR_RGB2GRAY)
    v = cv2.medianBlur(v, 5)
    _, _mask = cv2.threshold(v, -1, 255, cv2.THRESH_OTSU)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=5)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=5)
    _mask = ~_mask
    return _mask

def get_level(n, cnt):
    if (n % 2) == 0:
        return get_level((n // 2), cnt+1)
    else:
        return cnt

def get_mask(slide_name, magnification):
    slide_path = glob(f'/workspace/data3/A100_data/*/*/{slide_name}.svs')
    if len(slide_path) == 0:
        slide_path = glob(f'/workspace/data4/A100_data/*/*/{slide_name}.svs')
        if len(slide_path) == 0:
            slide_path = glob(f'/workspace/data3/A100_data/*/{slide_name}.svs')
    slide_path = slide_path[0]
    
    wsi_slide = openslide.OpenSlide(slide_path)
    mpp = wsi_slide.properties['aperio.MPP']
    if float(mpp) < 0.40:
        ORG_MAG = 40
    else:
        ORG_MAG = 20
    
    LEVEL = get_level(int(ORG_MAG // 1.25), 0)
    w_, h_ = wsi_slide.level_dimensions[LEVEL//2]
    wsi_array = np.array(wsi_slide.read_region([0, 0], LEVEL//2, [w_, h_]))
    if LEVEL % 2:
        wsi_array = cv2.resize(wsi_array, (w_//2, h_//2))
    otsu_mask = get_otsu_mask(wsi_array)
    gray_mask = get_gray_mask(wsi_array)
    tissue_mask = cv2.bitwise_or(otsu_mask, gray_mask)
    
    LEVEL = get_level(int(ORG_MAG // magnification), 0)
    w_, h_ = wsi_slide.level_dimensions[LEVEL//2]
    if LEVEL % 2:
        w_, h_ = w_//2, h_//2
    tissue_mask = cv2.resize(tissue_mask, (w_, h_))
    
    return tissue_mask, mpp, LEVEL

In [3]:
sample = ['A100_00206_03_stm_c1.png','A100_01244_02_en_c1.png',
'A100_00727_01_col_c1.png','A100_02301_14_stm_c1.png',
'A100_00408_01_stm_c1.png','A100_02139_08_gu_c1.png',
'A100_01597_05_gu_c1.png','A100_01602_09_gu_c1.png',
'A100_02070_02_gu_c1.png','A100_01564_10_gu_c1.png',
'A100_01498_12_gu_c1.png','A100_01600_05_gu_c1.png',
'A100_02953_10_col_c1.png','A100_00277_01_stm_c1.png',
'A100_00249_03_stm_c1.png','A100_02524_24_brs_c1.png',
'A100_01616_09_gu_c1.png','A100_00450_02_brs_c1.png',
'A100_00583_04_col_c1.png','A100_01650_17_stm_c1.png',
'A100_02524_01_brs_c1.png','A100_00210_21_stm_c1.png',
'A100_02850_08_col_c1.png','A100_02467_09_brs_c1.png',
'A100_01575_05_gu_c1.png','A100_01674_02_en_c1.png',
'A100_02054_05_gu_c1.png','A100_02400_05_brs_c1.png',
'A100_02401_19_brs_c1.png','A100_02356_03_stm_c1.png',
'A100_01546_09_gu_c1.png','A100_00940_13_bp_c1.png',
'A100_02506_03_brs_c1.png','A100_01773_02_en_c1.png',
'A100_01046_10_bp_c1.png','A100_02319_04_stm_c1.png',
'A100_02311_05_stm_c1.png','A100_01926_14_stm_c1.png',
'A100_01941_13_stm_c1.png','A100_00202_05_stm_c1.png']

In [5]:
for i in range(len(sample)):
    file_name = sample[i][:-4]
    tissue_mask, mpp, level = get_mask(file_name, 5)

    read_path = '/workspace/data4/changwoo/SDP/mask/Carcinoma_v4/5X/'+sample[i]
    img = cv2.imread(read_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.bitwise_and(img_gray, tissue_mask)
    img_blur = cv2.GaussianBlur(img_gray, (0, 0), 5)
    thres = int(0.5 * 255)
    _, thresh = cv2.threshold(img_blur, thres, 255, cv2.THRESH_BINARY_INV)
    contours, hierarchy = cv2.findContours(image = thresh, mode = cv2.RETR_TREE, method = cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)

    # zone size 이용
    img_copy = img.copy()
    l = []
    for i in range(1, len(contours)):
        if len(contours[i]) > 4:
            (x, y), (h, w), angle = cv2.fitEllipse(contours[i])
            if h > w:
                l.append(h)
            else:
                l.append(w)
    num = len(contours)
    max_length = max(l)
    max_length_mm = max_length * 0.002

    c = []
    for i in range(len(hierarchy[0])):
        if hierarchy[0][i][3] == 0:
            c.append(contours[i])
    c_1d = np.array(c)
    contours = c_1d

    total_area = 0
    area = []
    for i, cnt in enumerate(contours):
        a = cv2.contourArea(cnt)
        total_area = total_area + a
        area.append(a)
    max_area = max(area)

    _, tissue_thresh = cv2.threshold(tissue_mask, 0, 255, cv2.THRESH_BINARY_INV)
    # Find the contour of the figure 
    tissue_contours, hierarchy = cv2.findContours(image = tissue_thresh, mode = cv2.RETR_TREE, method = cv2.CHAIN_APPROX_SIMPLE)
    tissue_area = 0
    for i, cnt in enumerate(tissue_contours):
        a = cv2.contourArea(cnt)
        tissue_area = tissue_area + a
    if tissue_area == 0:
        percent = 0
    else:
        percent = total_area/tissue_area*100

    final = [file_name, num, total_area, percent, max_length, max_length_mm, max_area]
    print(final)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


['A100_00206_03_stm_c1', 147, 213584.5, 0.12311044464229363, 152.0731964111328, 0.30414639282226563, 8599.0]
['A100_01244_02_en_c1', 1258, 1866683.5, 1.0056319035934673, 239.74095153808594, 0.4794819030761719, 13623.5]
['A100_00727_01_col_c1', 170, 271819.5, 0.10989163327664173, 154.65806579589844, 0.3093161315917969, 8235.5]
['A100_02301_14_stm_c1', 442, 709731.5, 0.41055551364569676, 236.45925903320312, 0.4729185180664063, 12646.0]
['A100_00408_01_stm_c1', 331, 512597.5, 0.30907943725778325, 160.41867065429688, 0.32083734130859376, 10521.0]
['A100_02139_08_gu_c1', 14, 20808.0, 0.018884604881300077, 110.4068832397461, 0.2208137664794922, 6530.5]
['A100_01597_05_gu_c1', 20, 26087.5, 0.03066980647108169, 93.01307678222656, 0.18602615356445312, 4030.0]
['A100_01602_09_gu_c1', 18, 15630.0, 0.008996075552691572, 114.8480453491211, 0.2296960906982422, 3389.0]
['A100_02070_02_gu_c1', 16, 33571.5, 0.017685685160914674, 126.27304077148438, 0.25254608154296876, 6311.5]
['A100_01564_10_gu_c1', 1